<a href="https://www.kaggle.com/code/nigamshitij/parse-book-reviews-per-book?scriptVersionId=193072392" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import requests
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime
import time
import random
import re

In [10]:
def remove_boilerplate(text):
    boilerplate = ['cookie policy', 'privacy policy', 'terms of service', 'all rights reserved', '\n']
    for phrase in boilerplate:
        text = re.sub(r'(?i)' + re.escape(phrase) + r'.*', '', text)
    return text

In [11]:
def truncate_at_comments(review_text):
    comment_identifiers = [
        'Comments', 'Leave a comment', 'Reader comments', 
        'What do you think?', 'Join the discussion'
    ]
    
    lower_text = review_text.lower()
    for identifier in comment_identifiers:
        index = lower_text.find(identifier.lower())
        if index != -1:
            return review_text[:index].strip()
    
    return review_text

In [14]:
def remove_comments(review_soup):
    # Common class names for comment sections
    comment_classes = ['comment', 'comments-list', 'comments-area', 'comments', 'comment-section', 'user-comments', 'disqus_thread']
    
    for class_name in comment_classes:
        comment_section = review_soup.find('div', class_=class_name)
        if comment_section:
            comment_section.decompose()  # This removes the element from the soup
    
    return review_soup

In [17]:
def search_book_reviews(book_name, author):
    # Combine book name and author for search query
    search_query = f"{book_name} {author} book review -site:goodreads.com -site:amazon.* -site:reddit.com -site:barnesandnoble.com -site:wikipedia.*&start=10"
    
    # Perform Google search
    url = f"https://www.google.com/search?q={search_query.replace(' ', '+')}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    print(f"URL being looked at: {url}")
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract search results
    search_results = soup.find_all('div', class_='g')
    
    reviews = []
    
    for result in search_results:
        link = result.find('a')
        if link:
            review_url = link['href']

            # Skip non-http links and known non-review sites
            if not review_url.startswith('http') or any(site in review_url for site in ['google.com', 'wikipedia.org']):
                continue
            
            try:
                # Fetch the review page
                review_response = requests.get(review_url, headers=headers, timeout=10)
                review_soup = BeautifulSoup(review_response.text, 'html.parser')
                
                # Remove Comments
                review_soup = remove_comments(review_soup)
                
                # Extract review text (this is a simplistic approach and may need refinement)
                review_paragraphs = review_soup.find_all('main')
                # review_list_items = review_soup.find_all('li')
                
                # Search for relevant text
                review_text = ' '.join([p.text for p in review_paragraphs if re.search(r'\b(review|book|read|author|story|character)\b', p.text, re.IGNORECASE)])
                
                # Remove boilerplate
                review_text = remove_boilerplate(review_text)
                
                # Truncate at Comments
                review_text = truncate_at_comments(review_text)
                
                # original
                # review_text = ' '.join([p.text for p in review_soup.find_all('p')])
                
                # Extract date (this is a placeholder, as date formats vary widely)
                date = review_soup.find('time')
                review_date = date['datetime'] if date else 'Unknown'
                
                reviews.append({
                    'review_text': review_text[:10000],  # Limit to first 500 (original) characters
                    'review_date': review_date,
                    'review_website': review_url
                })
                
            except Exception as e:
                print(f"Error processing {review_url}: {str(e)}")
            
            # Be polite to servers
            time.sleep(random.uniform(1, 3))
    
    # Create DataFrame
    df = pd.DataFrame(reviews)
    
    return df

# Example usage
book_name = "Dune Book 1"
author = "Frank Herbert"

df = search_book_reviews(book_name, author)

# Save to CSV
df.to_csv(f"{book_name.replace(' ', '_')}_reviews.csv", index=False)
print(f"Reviews saved to {book_name.replace(' ', '_')}_reviews.csv")

URL being looked at: https://www.google.com/search?q=Dune+Book+1+Frank+Herbert+book+review+-site:goodreads.com+-site:amazon.*+-site:reddit.com+-site:barnesandnoble.com+-site:wikipedia.*&start=10
Error processing https://www.grimdarkmagazine.com/review-dune-by-frank-herbert/: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Reviews saved to Dune_Book_1_reviews.csv


In [18]:
df.head()

,review_text,review_date,review_website
0,On the desert planet... Illustration: Robert ...,Unknown,https://www.theguardian.com/books/2015/jul/03/...
1,,Unknown,https://brandeishoot.com/2024/02/09/reviewing-...
2,\n\n\n\n\n\n\n\nUnpopular Opinions on a Sci-Fi...,2020-10-20T21:33:00+11:00,https://ashsinfinitelibrary.wordpress.com/2020...
3,,Unknown,https://www.quora.com/Is-the-first-Dune-book-t...
4,\n\n\n\n\n\n\n\n\n\n\n\n\n\nDune by Frank Herb...,12 Nov,https://www.louiseslittlelife.com/blog/dune-by...


In [ ]:
add